In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [2]:
x = ['너', '임마', '밥은', '먹고', '다니냐']
n_grams = set(zip(*[x[i:] for i in range(2)]))
n_grams

{('너', '임마'), ('먹고', '다니냐'), ('밥은', '먹고'), ('임마', '밥은')}

In [3]:
generate_bigrams(['너', '임마', '밥은', '먹고', '다니냐'])

['너', '임마', '밥은', '먹고', '다니냐', '밥은 먹고', '임마 밥은', '너 임마', '먹고 다니냐']

# 데이터 전처리

In [4]:
import pandas as pd

In [5]:
columns = ['id', 'text', 'label']

train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', names=columns, skiprows=1).dropna()
test_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t', names=columns, skiprows=1).dropna()

In [6]:
train_data.to_csv('C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis\\train_data.csv', index = False)
test_data.to_csv('C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis\\test_data.csv', index = False)

In [7]:
import torch
from torchtext import data
from torchtext import datasets # 굳이 쓴 이유는??

In [8]:
SEED = 1234

torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cuda':
    torch.cuda.manual_seed_all(SEED)

In [9]:
from eunjeon import Mecab
mecab = Mecab()

In [10]:
# preprocessing : 전처리는 토큰화 후, 수치화하기 전 사이에서 작동
TEXT = data.Field(tokenize=mecab.morphs, preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype=torch.float)

In [11]:
fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}

In [12]:
train_data, test_data = data.TabularDataset.splits(
                            path = 'C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)

In [13]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [14]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = 'fasttext.simple.300d', unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [15]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    device = device,
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True)

In [16]:
next(iter(train_iterator)).text

tensor([[   71,     0,     0,  ...,  3664,    63,   132],
        [   10,    15,   119,  ...,    93,    63,   131],
        [    5,  1080,   385,  ...,    26,  1013,   252],
        ...,
        [    0,  2506, 16561,  ..., 17134,     0,  6983],
        [  628,     0,     0,  ...,     0,     0,  1002],
        [  135,   834,     0,  ..., 10950,     0,   981]], device='cuda:0')

In [17]:
TEXT.vocab.itos[2533]

'을 잃'

In [18]:
import torch.nn as nn
import torch.nn.functional as F

In [19]:
def print_shape(name, data):
    print(f'{name} has shape {data.shape}')

### avg_pool2d란?

In [20]:
nn.functional.avg_pool2d?

In [21]:
txt = torch.rand(2, 5, 10) # rand : 임의의 초기화된 행렬 생성
txt.shape, F.avg_pool2d(txt, (5, 1)).shape

(torch.Size([2, 5, 10]), torch.Size([2, 1, 10]))

In [22]:
txt = torch.tensor(
    [[[1,2,3,4],[4,5,6,7]]], dtype = torch.float)
print(txt.shape, '\t', txt)

torch.Size([1, 2, 4]) 	 tensor([[[1., 2., 3., 4.],
         [4., 5., 6., 7.]]])


In [23]:
# (2 * 1) 필터로 평균을 취함
F.avg_pool2d(txt, (2,1)).shape, F.avg_pool2d(txt, (2,1))

(torch.Size([1, 1, 4]), tensor([[[2.5000, 3.5000, 4.5000, 5.5000]]]))

In [24]:
F.avg_pool2d(txt, (2, 2)).shape, F.avg_pool2d(txt, (2, 2))

(torch.Size([1, 1, 2]), tensor([[[3., 5.]]]))

# FastText 모델 구현

In [25]:
class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        
        # permute : 모든 차원의 순서를 재배치
        # permute(2, 1, 0) # 0->2, 1->1, 2->0으로 차원변경
        embedded = embedded.permute(1,0,2)

        pooled = F.avg_pool2d(embedded, (embedded.shape[1],1)).squeeze(1)
        
        res  = self.fc(pooled)
        return res

In [26]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [27]:
model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,} 개 입니다.')

모델의 파라미터 수는 7,500,901 개 입니다.


In [29]:
pretrain_weight = TEXT.vocab.vectors
print(pretrain_weight.shape, model.embedding.weight.data.shape)

torch.Size([25002, 300]) torch.Size([25002, 300])


In [30]:
model.embedding.weight.data.copy_(pretrain_weight)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [ 0.2542,  1.2173,  1.8023,  ..., -0.9746,  0.1054, -1.7293],
        [-0.1084, -0.5668,  0.1102,  ..., -0.5685,  1.6376,  0.2508],
        [-0.3535,  1.0225, -1.7970,  ...,  0.0683,  0.3403,  1.5236]])

In [31]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# 모델 훈련

In [32]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [33]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [34]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [35]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        # autograd를 사용하여 역전파 단계를 계산
        loss.backward()
        # Optimizer의 step 함수를 호출하면 매개변수가 갱신
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
        

In [45]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 모델 훈련

In [47]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.217 | Train Acc: 91.47%
	 Val. Loss: 0.400 |  Val. Acc: 85.36%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 0.209 | Train Acc: 91.73%
	 Val. Loss: 0.417 |  Val. Acc: 85.14%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 0.203 | Train Acc: 92.02%
	 Val. Loss: 0.433 |  Val. Acc: 85.02%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.198 | Train Acc: 92.22%
	 Val. Loss: 0.449 |  Val. Acc: 84.71%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.194 | Train Acc: 92.45%
	 Val. Loss: 0.463 |  Val. Acc: 84.62%


In [50]:
model.load_state_dict(torch.load('tut3-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.402 | Test Acc: 85.13%


In [53]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok for tok in mecab.morphs(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [54]:
predict_sentiment(model, "이 영화 진짜 재밌었다!!")

0.9997900128364563

In [55]:
predict_sentiment(model, "영화관에서 이걸 본 내가 바보다. 내 돈 돌려줘!!!")

1.6363233953597955e-05

In [56]:
predict_sentiment(model, "이 영화 감독 밥은 먹고 다니냐? 이런 영화 만들고 잠이 와?")

0.002621039282530546

In [57]:
predict_sentiment(model, "내 인생 영화 등극. 주인공한테 너무 몰입해서 시간 가는 줄도 몰랐다...")

0.9996906518936157